In [1]:
import findspark
findspark.init()

from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from operator import add

sc = SparkContext()
ssc = StreamingContext(sc, 10)  # 10 seconds batch interval

host = "localhost"
port = 5555
socket_stream = ssc.socketTextStream(host, port)

lines = socket_stream.window(20)

hashtags = lines.flatMap(lambda line: line.split())\
                .filter(lambda word: word.lower().startswith("#"))

pairs = hashtags.map(lambda hashtag: (hashtag, 1))
hashtags_counts = pairs.reduceByKey(add)

sorted_hashtags_counts = hashtags_counts.transform(
    lambda rdd: rdd.sortBy(lambda x: x[1], ascending=False)
)
    
sorted_hashtags_counts.pprint()

In [2]:
# start streaming
ssc.start()
ssc.awaitTermination()

-------------------------------------------
Time: 2018-11-07 17:59:10
-------------------------------------------
('#ElectionResults2018', 2)
('#Democrats', 1)
('#WednesdayWisdom', 1)
('#ElectionResults2…RT', 1)
('#Trump2020', 1)
('#MAGA2…RT', 1)
('#Trump', 1)
('#MidtermsElections2018', 1)
('#SpiceGirls', 1)
('#PoshSpice…', 1)
...

-------------------------------------------
Time: 2018-11-07 17:59:20
-------------------------------------------
('#Trump', 5)
('#Democrats', 2)
('#ElectionResults2018', 2)
('#MidtermsElections2018', 2)
('#Midterms2018', 2)
('#ElectionNight', 2)
('#BLEXIT', 1)
('#Bredesen', 1)
('#WalkAwayFromDemocratsRT', 1)
('#MiMandat…', 1)
...

-------------------------------------------
Time: 2018-11-07 17:59:30
-------------------------------------------
('#Trump', 4)
('#MAGA', 3)
('#ElectionNight', 2)
('#BLEXIT', 1)
('#Bredesen', 1)
('#WalkAwayFromDemocratsRT', 1)
('#MiMandat…', 1)
('#MakeAmeri…RT', 1)
('#Democrats', 1)
('#DNC', 1)
...



KeyboardInterrupt: 

In [3]:
# close the connection
ssc.stop()

-------------------------------------------
Time: 2018-11-07 17:59:40
-------------------------------------------
('#MAGA', 2)
('#Blog1600Penn', 1)
('#midterms2018RT', 1)
('#1', 1)
('#2', 1)
('#Saudi', 1)
('#4', 1)
('#Yemen', 1)
('#Trump:', 1)

